# Tokenisation, embedding & PCA
Ce jupyter notebook montre étape par étape comment produire une cartographie de prompt en ce basant sur l'encodeur CLIP.
Pour une liste de prompt définis, nous calculons leurs *embeddings* respectif pour ensuite le réduire à deux dimensions

In [ ]:
from sklearn.decomposition import PCA
from itertools import product
import numpy as np
import json

from transformers import AutoTokenizer, CLIPTextModelWithProjection

# Chargement du modèle de tokenisation et d'embedding depuis HG_HUB
model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")


## Creation du matrix prompt 
On produit des prompts suivant 3 figures : un sujet, des informations de lumières, et des informations sur du détails photographique. On combine ensuite ces bout de phrase pour produire les prompts.

In [3]:

def pipe_to_list(pipe:str):
    # Transforme un str de type "a|b" en liste [a, b]
    return pipe.split("|")

subjects = "A man | a woman | landscape forest | view from the exterior of a cavern | Beautiful landscape, sunset | Montain landscape | city seen from above the ground, skyscraper |   a person with a beautiful smile | crowd of people on the street | Small dog on a couch | a cat next to a window | a space alien "
lights = "Natural light | hard light | soft light "
photographics = "Intricate detail, ultra sharp | 50mm | 120mm | long shot | ISO 1000 | ISO 100 | long pose shot | portrait shot | close-up | ultra close-up | long distance shoot | fisheye | 8mm | 16m"

prompts = [pipe_to_list(subjects), pipe_to_list(lights), pipe_to_list(photographics)]



prompts_all = []
for item in product(*prompts):
    prompts_all.append(", ".join(item))
len(prompts_all) # => 504 prompts généré

504

In [4]:
# Tokenisation
inputs = tokenizer(prompts_all, padding=True, return_tensors="pt")
# Embedding
outputs = model(**inputs)
text_embeds = outputs.text_embeds

In [7]:
np_embeds = text_embeds.detach().numpy() # on transforme les embedding de tensor a numpy array
pca = PCA(n_components=2) # chargement du pca
pca.fit(np_embeds) # On module le PCA en fonction des données fournis.
array_2dim  = pca.transform(np_embeds) # On réduit les nombre de dimension des valeurs
array_2dim.shape #=> (504, 2) 

(504, 2)

## Enregistrement des données

Pour afficher les données on enregistre les donnés en json sous forme : 
`[{"prompt1" : [x1, y1]}, {"prompt2" : [x2, y2]}]`


In [9]:
json_parse = {}
for index, prompt in enumerate(prompts_all):
    json_parse[prompt] = array_2dim[index].tolist()

with open("myPCA.json", "w+", encoding="utf-8") as f:
    json.dump(json_parse, f, ensure_ascii=True, indent=2)